### A

In [3]:
inf = float('inf')

n, h, x = map(int, input().split())
li = list(map(int, input().split()))

min_r = inf
min_r_i = inf
for i in range(n):
    v = li[i]
    
    if v + h >= x:
        if v < min_r:
            min_r = v
            min_r_i = i + 1
            
print(min_r_i)

10 500 999
38 420 490 585 613 614 760 926 945 999
4


### B

In [6]:
n = int(input())
li = sorted(map(int, input().split()))

r = -1
for i in range(n - 1):
    if li[i] + 1 != li[i + 1]:
        r = li[i] + 1
        break
        
print(r)

16
152 153 154 147 148 149 158 159 160 155 156 157 144 145 146 150
151


### C

In [14]:
from itertools import permutations as perm
import io, os, sys
input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

inf = float('inf')

n, m = map(int, input().split())

dists = [[-inf] * n for _ in range(n)]
for _ in range(m):
    a, b, c = map(int, input().split())
    a -= 1 
    b -= 1
    
    dists[a][b] = max(dists[a][b], c)
    dists[b][a] = max(dists[b][a], c)
    
max_dist = 0
for length in range(2, n + 1):
    for case in perm(range(n), length):
        dist = 0
        for i in range(length - 1):
            v = case[i]
            w = case[i + 1]
            
            dist += dists[v][w]
            
        max_dist = max(max_dist, dist)
            
print(max_dist)

4 4
1 2 1
2 3 10
1 3 100
1 4 1000
1110


### D

In [17]:
import io, os, sys
input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

inf = float('inf')

n = int(input())
li = [list(map(int, input().split())) for _ in range(n)]

nli = []
all_z = 0
t_z = 0
for i in range(n):
    x, y, z = li[i]
    all_z += z
    
    if x > y:
        t_z += z
    else:
        diff = y - x
        needed = (diff + 1) // 2
        
        nli.append((needed, z))
        
a_z = all_z - t_z

if t_z > a_z:
    print(0)
else:
    diff = a_z - t_z
    needed = (diff + 1) // 2
    
    dp = [inf] * (all_z + 1)
    dp[0] = 0
    for i in range(len(nli)):
        to_change, z = nli[i]
        
        for z_i in range(len(dp) - 1, z - 1, -1):
            dp[z_i] = min(dp[z_i], dp[z_i - z] + to_change)
            
    result = min(dp[needed:])
    
    print(result)

3
3 4 2
1 2 3
7 2 6
0


### E

In [ ]:
from collections import deque
import sys
input = lambda: sys.stdin.readline().rstrip()

inf = float('inf')

h, w = map(int, input().split())
li = [list(input()) for _ in range(h)]

obs = '#><v^'

is_seens = [[False] * w for _ in range(h)]

sx, sy = -1, -1
ex, ey = -1, -1
for i in range(h):
    # > checker
    is_seen = False
    for j in range(w):
        if li[i][j] in obs:
            is_seen = False
            if li[i][j] == '>':
                is_seen = True
        else:
            if is_seen:
                is_seens[i][j] = True
                
        if li[i][j] == 'S':
            sx, sy = i, j
            li[i][j] = '.'
        elif li[i][j] == 'G':
            ex, ey = i, j
            li[i][j] = '.'
                
    # < checker
    is_seen = False
    for j in range(w - 1, -1, -1):
        if li[i][j] in obs:
            is_seen = False
            if li[i][j] == '<':
                is_seen = True
        else:
            if is_seen:
                is_seens[i][j] = True
                
for j in range(w):
    # v checker
    is_seen = False
    for i in range(h):
        if li[i][j] in obs:
            is_seen = False
            if li[i][j] == 'v':
                is_seen = True
        else:
            if is_seen:
                is_seens[i][j] = True
                
    # ^ checker
    is_seen = False
    for i in range(h - 1, -1, -1):
        if li[i][j] in obs:
            is_seen = False
            if li[i][j] == '^':
                is_seen = True
        else:
            if is_seen:
                is_seens[i][j] = True 
                
dists = [[inf] * w for _ in range(h)]
dists[sx][sy] = 0

queue = deque()
queue.append((sx, sy))

dx = [-1, 1, 0, 0]
dy = [0, 0, -1, 1]

while queue:
    x, y = queue.popleft()
    
    for d in range(4):
        nx = x + dx[d]
        ny = y + dy[d]
        
        if nx < 0 or nx >= h or ny < 0 or ny >= w:
            continue
        if li[nx][ny] in obs or is_seens[nx][ny]:
            continue
        if dists[nx][ny] == inf:
            dists[nx][ny] = dists[x][y] + 1
            queue.append((nx, ny))
            
if dists[ex][ey] == inf:
    print(-1)
else:
    print(dists[ex][ey])

### G

In [ ]:
[c, a, b] a번째 행에 b가 c개 있다.

In [40]:
# 틀렸음
import io, os, sys
# input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

from collections import Counter
# https://github.com/tatyam-prime/SortedSet/blob/main/SortedSet.py
import math
from bisect import bisect_left, bisect_right
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedSet(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedSet from iterable. / O(N) if sorted and unique / O(N log N)"
        a = list(a)
        if not all(a[i] < a[i + 1] for i in range(len(a) - 1)):
            a = sorted(set(a))
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedSet" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def add(self, x: T) -> bool:
        "Add an element and return True if added. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return True
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i != len(a) and a[i] == x: return False
        a.insert(i, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()
        return True

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True
    
    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError
    
    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return 

# n, m = map(int, input().split())
# li = [list(map(int, input().split())) for _ in range(n)]

n, m = 6, 3
nums = []
for i in range(1, n + 1):
    nums.extend([i] * m)
    
from random import shuffle
shuffle(nums)

li = [[0] * m for _ in range(n)]
for i in range(n):
    for j in range(m):
        li[i][j] = nums.pop()
        
for v in li:
    print(v)

sst = SortedSet()
for i in range(n):
    ct = Counter(li[i])
    
    for key in ct:
        count = ct[key]
        
        sst.add([count, i, key])
        
result = [[0] * m for _ in range(n)]
for j in range(m):
    filled_is = set()
    used_nums = set()
    
    temp = []
    for sst_i in range(len(sst) - 1, -1, -1):
        count, i, key = sst[sst_i]
        
        if i not in filled_is and key not in used_nums:
            result[i][j] = key
            filled_is.add(i)
            used_nums.add(key)
            
            sst.discard(sst[sst_i])
            if count - 1 >= 1:
                temp.append([count - 1, i, key])
                
    print(result, sst)
            
    for tt in temp:
        sst.add(tt)
        
print('Yes')
for v in result:
    print(*v)

[1, 6, 3]
[4, 5, 1]
[3, 2, 1]
[2, 5, 5]
[6, 3, 2]
[6, 4, 4]
[[0, 0, 0], [1, 0, 0], [3, 0, 0], [5, 0, 0], [6, 0, 0], [4, 0, 0]] {[1, 0, 1], [1, 0, 3], [1, 0, 6], [1, 1, 4], [1, 1, 5], [1, 2, 1], [1, 2, 2], [1, 3, 2], [1, 4, 2], [1, 4, 3], [1, 5, 6]}
[[0, 1, 0], [1, 4, 0], [3, 2, 0], [5, 5, 0], [6, 3, 0], [4, 6, 0]] {[1, 0, 3], [1, 0, 6], [1, 1, 5], [1, 2, 1], [1, 3, 2], [1, 4, 2], [1, 5, 4]}
[[0, 1, 6], [1, 4, 5], [3, 2, 1], [5, 5, 0], [6, 3, 2], [4, 6, 4]] {[1, 0, 3], [1, 3, 2]}
Yes
0 1 6
1 4 5
3 2 1
5 5 0
6 3 2
4 6 4


In [41]:
sst

SortedSet[[[1, 0, 3], [1, 3, 2]]]